In [1]:
import numpy as np
from src.monte_carlo import SimulationVisualizer, DirectorySimulationVisualizer
from src.utils import create_list_covariance_matrices
from functools import partial
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
from ipywidgets import fixed, interact_manual

In [2]:
# Default data construction parameters
construct_kwds = {
    "y_sd": [1,1],
    "drought_mean": [0, 0],
    "drought_cov": [1,1],
    "beta0": [1,2],
    "beta1": [-1,-2],
    "y_name": "y",
    "weight": 0.1,
    "reg_ready": False,
    "output_true_beta": True,
}

def STN(sigma, beta0, beta1, return_single = False, return_other = False):
    
    A = np.array(beta0)/np.sqrt(np.array(beta1)**2 + np.array(sigma)**2)
    
    if return_single:
        if return_other:
            return A[1]
        else:
            return A[0]
    
    return A

STN_partial = partial(STN,
                      beta0 = construct_kwds.get('beta0'),
                      beta1 = construct_kwds.get('beta1'),
                      return_single=True)

# create multivariate signal to noise ratio based on mahalanobis distance
def STN_m(sigma, beta0, beta1):
    """Calculate multivariate STN

    Args:
        mu (np.ndarray): vector of means
        sigma (np.ndarray): variance-covariance matrix
    """
    # First get inverse of var-covar
    if isinstance(sigma, list):
        sigma = np.array(sigma)
    if isinstance(beta0, list):
        beta0 = np.array(beta0)
    if isinstance(beta1, list):
        beta1 = np.array(beta1)
            
    
    sigma_2 = beta1@sigma@beta1 + np.identity(beta1.size)
    
    sigma_inv = numpy.linalg.inv(sigma_2)
    
    return mahalanobis(beta0, 0, sigma_inv)

STN_m_partial = partial(STN_m,
                        beta0 = construct_kwds.get('beta0'),
                        beta1 = construct_kwds.get('beta1'))

STN_partial_other = partial(STN,
                      beta0 = construct_kwds.get('beta0'),
                      beta1 = construct_kwds.get('beta1'),
                      return_single=True,
                      return_other=True)

construct_kwds_3 = {
    "y_sd": [1,1,1],
    "drought_mean": [0, 0,0],
    "drought_cov": [1,1,1],
    "beta0": [1,2,3],
    "beta1": [-1,-2,-3],
    "y_name": "y",
    "weight": 0.1,
    "reg_ready": False,
    "output_true_beta": True,
}

STN_partial_3 = partial(STN,
                      beta0 = construct_kwds_3.get('beta0'),
                      beta1 = construct_kwds_3.get('beta1'),
                      return_single=True)


def get_covar(x):
    return np.array(x)[0,1]



In [ ]:
v = DirectorySimulationVisualizer()

: 

In [ ]:
y_sd_sym = v.create_data(
    directory = 'y_sd_sym_two_weights_new',
    param_name='y_sd',
    other_var='weight',
    f = STN_partial,
    num_replace=1,
    regex=False,
    )

y_sd_sum_stats = v.calculate_statistics(y_sd_sym, other_var=True)

v.plot(y_sd_sum_stats,to_plot='sigma', level_subset= ('other_var', 0.1))

: 

In [ ]:

drought_shocks_2 = v.create_data(
    directory='drought_shocks_two_weights_new',
    param_name='drought_cov',
    other_var='weight',
    f = get_covar,
    regex=True,
    newline_repl=True
    )

drought_cov_shocks = v.calculate_statistics(drought_shocks_2, other_var=True, only_success=True)
v.plot(drought_cov_shocks,to_plot='beta_1', level_subset= ('other_var', 0.1))





: 

In [ ]:
v3 = DirectorySimulationVisualizer(regimes=3)


y_sd_sym_3 = v3.create_data(
    param_name='y_sd',
    directory='y_sd_sym_3_two_weights_new2',
    other_var='weight',
    f = STN_partial_3,
    regex=True,
    )

y_sd_sym_3_stats = v3.calculate_statistics(y_sd_sym_3, other_var=True, only_success=True)

v3.plot(y_sd_sym_3_stats, to_plot='beta_1', level_subset= ('other_var', 0.1))



: 

In [ ]:
beta1_diff_data = v.create_data(
    param_name='beta1',
    directory = 'beta1_diff_two_weights_new',
    other_var='weight',
    f = lambda x: x[1],
    comma_repl=True,
    num_replace=2,
    )
beta1_diff_stats = v.calculate_statistics(beta1_diff_data,  other_var=True)
v.plot(beta1_diff_stats, to_plot='beta_1', level_subset= ('other_var', 0.1))

: 

In [ ]:

drought_shock_3_data = v3.create_data(
    param_name='drought_cov',
    directory = 'drought_shocks_3_two_weights_new2',
    other_var='weight',
    f = lambda x: np.array(x)[0,1],
    regex=True,
    matrix_repl=True)

drought_shock_3_data.loc[:, (['success'], slice(None), slice(None))]['success']['Unnamed: 28_level_1']['Unnamed: 28_level_2'].value_counts()
drought_shock_3_stats = v3.calculate_statistics(drought_shock_3_data, other_var=True, only_success=True)
v3.plot(drought_shock_3_stats, to_plot='beta_1', level_subset = ('other_var', 0.1))

: 

In [ ]:
beta_1_diff_3_data = v3.create_data(
    directory='beta1_diff_3_two_weights_new2',
    param_name='beta1',
    other_var='weight',
    f = lambda x: x[1],
    regex=True,
    newline_repl=True)

beta_1_diff_3_stats = v3.calculate_statistics(beta_1_diff_3_data, other_var=True)

v3.plot(beta_1_diff_3_stats, to_plot='beta_1', level_subset=('other_var', 0.1))

: 

## Misclassification 

In [ ]:
misclass_2_data = v.create_data(
    directory='misclass_2_test',
    param_name='weight',
    f = None,
    str_replace=False
    )

misclass_2_stats = v.calculate_statistics(misclass_2_data)

v.plot(misclass_2_stats, to_plot='beta_1')

: 

In [ ]:
misclass_3_data = v3.create_data(
    directory='misclass_3_test',
    param_name='weight',
    f = None,
    str_replace=False)

misclass_3_stats = v3.calculate_statistics(misclass_3_data, only_success=True)

v3.plot(misclass_3_stats, to_plot='beta_1', xlabel = 'Misclassification')

: 

In [ ]:
data_dict = {k:v for k,v in zip(['sigma_2', 'sigma_3', 'drought_2', 'drought_3', 'beta1_diff'], [y_sd_sym, y_sd_sym_3, drought_shocks_2, drought_shock_3_data, beta1_diff_data])}

: 

In [ ]:
@interact_manual(df=list(data_dict.keys()), param=['beta_1', 'beta_0', 'sigma'], weight=[0.1, 0.7], regimes=[2,3])
def show_plots(df, param, weight, regimes):

    v = DirectorySimulationVisualizer(regimes=regimes)

    data_stats = v.calculate_statistics(data_dict.get(df), other_var=True, only_success=True)

    v.plot(data_stats, to_plot=param, level_subset= ('other_var', weight))

: 

In [ ]:
show_plots(data_list[0], 'beta_1', 0.7, 2)

: 